# Library Importation

In [1]:
# data maniuplation
import numpy as np
import pandas as pd
from functools import reduce
# web scraping
import requests
from bs4 import BeautifulSoup
# ssl certification
import ssl

# Constant and function definition

In [2]:
# ssl certification
ssl._create_default_https_context = ssl._create_unverified_context
# pre-define argument for pd.read_csv
arg_read_csv = {'header':1, 'skipfooter':3, 'engine':'python'}
# pre-defind argument for pd.merge
col_merge = ['date', 'region']
# 3 series for temperature (Mean, Max, Min)
data_series_temperature = ['CLMTEMP', 'CLMMAXT', 'CLMMINT']
# known stations for temperature
station_temperature = ['CCH','CWB','HKA','HKO','HKP','HKS','HPV','JKB','KLT','KP','KSC','KTG','LFS','NGP','PEN','PLC','SE1','SEK','SHA','SKG','SKW','SSH','SSP','STY','TC','TKL','TMS','TPO','TU1','TW','TWN','TY1','TYW','VP1','WGL','WLP','WTS','YCT']
# known stations for series other than temperature
staion_known = ['BHD','CCB','CCH','CPH','CP1','CWB','GI','HKA','HKO','HKP','HKS','HPV','JKB','KLT','KP','KSC','KTG','LAM','LFS','NGP','NP','PEN','PLC','SC','SE','SE1','SEK','SF','SHA','SHL','SKG','SKW','SSH','SSP','STY','TC','TKL','TME','TMS','TPK','TPO','TUN','TU1','TW','TWN','TY1','TYW','VP1','WGL','WLP','WTS','YCT']

In [3]:
def get_link_list(series_id):
  '''
  Output: list of link of csv on the website
  Input:  data id (i.e. daily-maximum-mean-heat-index)
  Note: not suitable for temperature
  '''
  response = requests.get(f"https://data.gov.hk/tc-data/dataset/hk-hko-rss-{series_id}")
  soup = BeautifulSoup(response.text, 'html.parser')
  result = soup.find_all('a', "dataset-details__list-item-download btn btn--light")
  link_list = []
  for tag in result:
    if isinstance(link:=tag.get("href"), str) and "ALL" in link:
      link_list.append(link)
  return link_list

In [4]:
def digit_or_null(value):
    '''
    Output: float or numpy null value
    Input: single value (i.e. integer, float, string)
    Note: used in apply() of dataframe
    '''
    try:
        float(value)
    except:
        return np.nan
    else:
        return float(value)

In [5]:
def read_csv_temperature(link, year=0):
  '''
  Output: Pandas Series, date as index
  Input: link of csv, year of start
  Note: only for temperature 
  '''
  df = pd.read_csv(link, **arg_read_csv, sep=None)
  col_name = df.columns[0]
  df_temp = df.iloc[1:,0].str.split(",", expand=True)
  df_rename = df_temp.rename(columns={0: 'year', 1: 'month', 2: 'day', 3: col_name, 4:'completeness'})
  df_valid_data = df_rename[(df_rename['year'] != '1900') | (df_rename['month'] != '2') | (df_rename['day'] != '29')].reset_index()
  df_valid_data = df_valid_data.copy()
  df_valid_data['date'] = pd.to_datetime(df_valid_data[['year','month','day']])
  df_date = df_valid_data.set_index('date')
  df_date_filter = df_date[df_date.index.year >= year]
  return df_date_filter[col_name].apply(lambda x: digit_or_null(x))
  

In [6]:
def read_csv(link, year=0):
  '''
  Output: Pandas Series, date as index
  Input: link of csv, year of start
  Note: not suitable for temperature 
  '''
  df = pd.read_csv(link, **arg_read_csv)
  col_name = df.columns[0]
  df_temp = df.reset_index().iloc[1:].dropna(subset='level_3')
  df_rename = df_temp.rename(columns={'level_0': 'year', 'level_1': 'month', 'level_2': 'day', 'level_3': col_name, col_name:'completeness'})
  df_valid_data = df_rename[(df_rename['year'] != '1900') | (df_rename['month'] != '2') | (df_rename['day'] != '29')]
  df_valid_data = df_valid_data.copy()
  df_valid_data['date'] = pd.to_datetime(df_valid_data[['year','month','day']])
  df_date = df_valid_data.set_index('date')
  df_date_filter = df_date[df_date.index.year >= year]
  return df_date_filter[col_name].apply(lambda x: digit_or_null(x))


In [7]:
def melt(df):
  '''
  Output: DataFrame with 3 columns only, including "date", "region", "name of variable"
  Input: DataFrame with various columns, including "date", each region has one column
  Note: apply to dataframe of same series with various region
  '''  
  if isinstance(df, pd.Series):
    df = df.copy().to_frame()
  df_melt = pd.melt(df.reset_index(), id_vars=['date'])
  df_melt['region'] = df_melt['variable'].apply(lambda x: x.split("- ")[1] if "-" in x else x.split("at the ")[1]).str.rstrip()
  df_melt['type'] = df_melt['variable'].apply(lambda x: x.split(" -")[0] if "-" in x else x.split(" at the")[0].split("Daily ")[1])
  col_name = df_melt['type'].value_counts().index[0]
  return df_melt[['date', 'region', 'value']].rename(columns={'value':col_name})

In [8]:
def merge(list_df): 
    '''
    Output: DataFrame merged 
    Input: List of DataFrame with 3 columns only, including "date", "region", "name of variable"
    Note: Outer join by "date" and "region"
    '''     
    return reduce(lambda  left, right: pd.merge(left, right, on=col_merge, how='outer'), list_df)

In [9]:
def get_station_list(list_id):
    '''
    Output: List of station
    Input: List of series id
    Note: not suitable for temperature
    '''     
    return [link.split("/")[-1].split("_")[1] for id in list_id for link in get_link_list(id)]

In [10]:
def get_series_list(list_id):
    '''
    Output: List of series
    Input: List of series id
    Note1: not suitable for temperature
    Note2: one series id may contain more than one series. i.e. daily-maximum-mean-heat-index
    '''
    return [link.split("/")[-1].split("_")[2] for id in list_id for link in get_link_list(id)]

In [11]:
def unique_sort(list_input):
    '''
    Output: List
    Input: Duplicate-removed and Sorted list
    Note: sorted in alphabetical order
    '''    
    list_sorted = list(set(list_input))
    list_sorted.sort()
    return list_sorted

In [12]:
def get_temperature_station(series, station, year=0):
    '''
    Output: Cleaned Pandas Series by calling another function 
    Input: series (not series id), station in 3 letter, year in 4 digits
    Note: only for temperature
    '''     
    link = f"https://data.weather.gov.hk/weatherAPI/opendata/opendata.php?dataType={series}&rformat=csv&station={station}"
    return read_csv_temperature(link, year)

In [13]:
def get_wind_station(series, station, year=0):
    '''
    Output: Cleaned Pandas Series by calling another function 
    Input: series (not series id), station in 3 letter, year in 4 digits
    Note: only for wind
    ''' 
    link = f"https://data.weather.gov.hk/cis/csvfile/{station}/ALL/daily_{station}_{series}_ALL.csv"
    return read_csv(link, year)

In [14]:
def get_other_station(series, station, year=0):
    '''
    Output: Cleaned Pandas Series by calling another function 
    Input: series (not series id), station in 3 letter, year in 4 digits
    Note: not suitable for temperature and wind
    ''' 
    link = f"https://data.weather.gov.hk/weatherAPI/cis/csvfile/{station}/ALL/daily_{station}_{series}_ALL.csv"
    return read_csv(link, year)

In [15]:
def merge_all_station_in_all_series(list_series, list_station, func, year=0):
    '''
    Output: merged dataframe for all station in all series 
    Input: list of series (not series id), list of station, function to get series
    Note: separatedly called if the function to get series is different
    '''     
    list_series_all_station_all = []
    list_series_one_station_all = []
    for series in list_series:
        for station in list_station:
            try:
                result = func(series, station, year)
            except:
                pass
            else:
                list_series_one_station_all.append(result)
        list_series_all_station_all.append(melt(pd.concat(list_series_one_station_all, axis=1)))
        list_series_one_station_all = []
    return merge(list_series_all_station_all)

# All Combination between Data Series and Station

## Find All Station

In [16]:
# series id is copied from https://data.gov.hk/ , they have similar api link
list_id_wind = ["daily-prevailing-wind-direction",
                "daily-mean-wind-speed"]

In [17]:
# series id is copied from https://data.gov.hk/ , they have similar api link
list_id_other = ["daily-maximum-mean-heat-index", 
                "daily-mean-amount-of-cloud", 
                "daily-mean-pressure",
                "daily-total-rainfall",
                "daily-mean-relative-humidity",
                "daily-global-solar-radiation",
                "daily-maximum-mean-uv-index",
                "daily-total-bright-sunshine"]

In [18]:
# get all possible stations from wind series
station_wind = get_station_list(list_id_wind)

In [19]:
# get all possible stations from other series
station_other = get_station_list(list_id_other)

In [20]:
# combine all possible stations from all series
station_all = station_other + station_temperature + station_wind + staion_known

In [21]:
# remove duplicate and sort it
station_unique_sort = unique_sort(station_all)

## Find All Data Series

In [22]:
# get all possible wind series from list of wind series id
data_series_wind = unique_sort(get_series_list(list_id_wind))

In [23]:
# get all possible other series from list of other series id
data_series_other = unique_sort(get_series_list(list_id_other))

## Combine All Series in All Station

In [24]:
# all temperature series in all stations
df_temp = merge_all_station_in_all_series(data_series_temperature, station_unique_sort, get_temperature_station, 2014)

In [25]:
# all wind series in all stations
df_wind = merge_all_station_in_all_series(data_series_wind, station_unique_sort, get_wind_station, 2014)

In [26]:
# all other series in all stations
df_other = merge_all_station_in_all_series(data_series_other, station_unique_sort, get_other_station, 2014)

In [27]:
# combine all series in all stations
df_all = merge([df_other, df_wind, df_temp])

## Final Formatting

In [28]:
# re-arrange the sequence of columns
df_final = df_all[['date',
                   'region',
                   'Mean HKHI',
                   'Max HKHI',
                   'Mean Amount of Cloud (%)',
                   'Mean Pressure (hPa)',
                   'Total Rainfall (mm)',
                   'Mean Relative Humidity (%)',
                   'Maximum Temperature (°C)',
                   'Minimum Temperature (°C)',
                   'Mean Temperature (°C)',
                   'Global Solar Radiation (MJ/m&sup2;)',
                   'Max UV Indices(15-minute average)',
                   'Mean UV Indices(7 a.m. to 6 p.m.)',
                   'Total Bright Sunshine (hours)',
                   'Prevailing Wind Direction (°)',
                   'Mean Wind Speed (km/h)'
                   ]].sort_values(["date", "region"]).reset_index(drop=True)

In [29]:
df_final

,date,region,Mean HKHI,Max HKHI,Mean Amount of Cloud (%),Mean Pressure (hPa),Total Rainfall (mm),Mean Relative Humidity (%),Maximum Temperature (°C),Minimum Temperature (°C),Mean Temperature (°C),Global Solar Radiation (MJ/m&sup2;),Max UV Indices(15-minute average),Mean UV Indices(7 a.m. to 6 p.m.),Total Bright Sunshine (hours),Prevailing Wind Direction (°),Mean Wind Speed (km/h)
0,2014-01-01,Bluff Head,NaN,NaN,NaN,NaN,NaN,NaN,20.8,13.7,16.3,NaN,NaN,NaN,NaN,80.0,9.7
1,2014-01-01,Central Pier,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,80.0,7.0
2,2014-01-01,Cheung Chau,NaN,NaN,NaN,1019.0,0.0,50.0,20.6,12.3,15.7,NaN,NaN,NaN,NaN,10.0,13.7
3,2014-01-01,Cheung Chau Beach,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350.0,9.0
4,2014-01-01,Ching Pak House(Tsing Yi),NaN,NaN,NaN,NaN,0.0,41.0,21.1,12.8,16.4,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208931,2024-12-31,Tuen Mun Government Offices,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,7.5
208932,2024-12-31,Waglan Island,NaN,NaN,NaN,1019.2,0.0,64.0,23.8,17.0,19.0,NaN,NaN,NaN,NaN,350.0,14.8
208933,2024-12-31,Wetland Park,NaN,NaN,NaN,1019.3,0.0,60.0,25.8,13.7,19.0,NaN,NaN,NaN,NaN,40.0,1.5
208934,2024-12-31,Wong Chuk Hang,NaN,NaN,NaN,NaN,NaN,55.0,23.7,14.2,20.0,NaN,NaN,NaN,NaN,130.0,5.3


## Save as csv

In [30]:
# save it in csv
df_final.to_csv("data_final.csv")